# Sistema de Modelado y Análisis de Supervivencia (médica)

En este notebook se detalla un sistema o secuencia de pasos a seguir que se pueden aplicar a cualquier modelo de Machine Learning que involucren problemas de tipo ***supervivencia***.

## 1. Dataset
* Cada fila representa a un paciente.
* El objetivo es predecir el riesgo (**risk**) de que un paciente muera
    * Esto se determina en la columna **event**.
---
En los análisis de supervivencia tenemos:
* Los targets:
    * **event**: True/False (muere/no muere).
    * **time**: El tiempo cuando el evento ocurre.
* Explanatory variables:
    * El resto de variables (age, prior_therapy, etc)

Por lo cual la formula de la ecuación matemática del modelo será:
$$
risk = (w_0) + (w_1) \cdot age + (w_2) \cdot prior\_therapy
$$

Con los modelos de Machine Learning, se busca encontrar los mejores valores (optimización) para los pesos ***w1*** y ***w2*** de la ecuación anterior, para finalmente calcular el **risk** asociado.

In [1]:
import pandas as pd

df_patients = pd.read_excel("../data/data_lung_cancer_smote.xlsx")
list_columns_categorical = df_patients.select_dtypes(include="object").columns
df_patients[list_columns_categorical] = df_patients[list_columns_categorical].astype("category")        # Transformación Object a Category. Paso importante para que OneHotEncoder() reconozca las variables categóricas y las transforme.
df_patients

,event,time,age,karnofsky_score,months_from_diagnosis,prior_therapy,treatment,celltype
0,True,2.373626,69.000000,60.000000,7.000000,No,Standard,Squamous
1,True,7.516484,38.000000,60.000000,3.000000,No,Standard,Squamous
2,True,4.153846,63.000000,60.000000,9.000000,Yes,Standard,Squamous
3,True,3.890110,65.000000,70.000000,11.000000,Yes,Standard,Squamous
4,True,0.329670,49.000000,20.000000,5.000000,No,Standard,Squamous
...,...,...,...,...,...,...,...,...
238,False,3.142881,65.810046,64.640822,4.762009,No,Standard,Smallcell
239,False,3.380047,36.495508,70.684273,21.551683,Yes,Test,Smallcell
240,False,3.082424,65.029553,81.087920,4.852974,No,Standard,Squamous
241,False,2.986648,62.424988,77.842548,4.084998,No,Standard,Smallcell


## 2. Feature Selection

Selección de las variables a utilizar en el modelo:
* `y (target)`: **event** y **time**
    * Para que estas variables puedan ser procesadas por el modelo, deben transformarse a otra estructura de datos (*numpy records array*). Esto se hace con *.to_records()*
* `x (explanatory)`: Variables relevantes para calcular el riesgo de un paciente.

### 2.1 Preprocessing Data

1. Revisar **NaN**: Eliminarlos del dataset
2. Transformar los datos categóricos de las variables Exploratory (X) a numéricos con **OneHotEncoder()**
    * La variable target (y) no necesita transformación de categóricos a numéricos ya que cuando se aplica el algoritmo de ML con .fit() este hace la transformación de forma automática.

In [2]:
df_patients.isna().sum()

event                    0
time                     0
age                      0
karnofsky_score          0
months_from_diagnosis    0
prior_therapy            0
treatment                0
celltype                 0
dtype: int64

In [3]:
y = df_patients[["event", "time"]].to_records(index=False)

In [4]:
X = df_patients.drop(["event", "time"], axis=1)
X

,age,karnofsky_score,months_from_diagnosis,prior_therapy,treatment,celltype
0,69.000000,60.000000,7.000000,No,Standard,Squamous
1,38.000000,60.000000,3.000000,No,Standard,Squamous
2,63.000000,60.000000,9.000000,Yes,Standard,Squamous
3,65.000000,70.000000,11.000000,Yes,Standard,Squamous
4,49.000000,20.000000,5.000000,No,Standard,Squamous
...,...,...,...,...,...,...
238,65.810046,64.640822,4.762009,No,Standard,Smallcell
239,36.495508,70.684273,21.551683,Yes,Test,Smallcell
240,65.029553,81.087920,4.852974,No,Standard,Squamous
241,62.424988,77.842548,4.084998,No,Standard,Smallcell


In [5]:
from sksurv.preprocessing import OneHotEncoder

In [6]:
encoder = OneHotEncoder()

In [7]:
X = encoder.fit_transform(X)

### 2.2 Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
pd.DataFrame({
    "Dataset": ["X_train", "X_test", "y_train", "y_test"],
    "Registros": [len(X_train), len(X_test), len(y_train), len(y_test)]
})

,Dataset,Registros
0,X_train,170
1,X_test,73
2,y_train,170
3,y_test,73


## 3. The Cox PH Model

### 3.1 Fit - Ajuste del modelo (ecuación matemática)

In [20]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

In [21]:
model_cox = CoxPHSurvivalAnalysis()

In [22]:
model_cox.fit(X_train, y_train)     # CORREGIR A FUTURO: Se está utilizando el dataset completo para entrenar, lo cual es un error. Se debe crear Train y Test sets.

CoxPHSurvivalAnalysis()

### 3.2 Predict - Predicciones con el modelo ajustado

In [23]:
model_cox.predict(X_test)

array([-2.42631351, -4.62376492, -2.39834002, -3.05017746, -3.01177653,
       -3.46572524, -2.87257884, -4.51907426, -3.77175433, -1.1503247 ,
       -2.94572483, -3.59776094, -2.6658907 , -2.8634695 , -1.59187046,
       -2.38605253, -3.47407139, -2.19553659, -4.62870278, -4.11147367,
       -2.75968436, -2.82470553, -1.20344857, -1.66147212, -4.47395023,
       -2.85080116, -3.01694016, -4.54121462, -3.16014558, -2.60014825,
       -2.6720411 , -4.06579712, -2.81959605, -4.71471842, -2.09981586,
       -4.12456341, -1.96065694, -4.57563361, -4.57302949, -4.67312082,
       -4.1682001 , -2.30957862, -1.67306527, -3.54951067, -4.40511172,
       -3.36221751, -3.42754093, -3.21441879, -2.13046766, -3.5295897 ,
       -3.22411751, -3.84117515, -4.26065335, -2.00373176, -2.6468285 ,
       -1.18449165, -2.72658573, -4.58756224, -4.23895649, -3.17387824,
       -2.49883838, -1.19322961, -4.6259645 , -3.28302233, -3.34961876,
       -2.27063555, -4.17053721, -3.8585503 , -4.26975827, -1.92

### 3.3 Score - Evaluación de las predicciones

In [24]:
model_cox.score(X_test, y_test)

0.756129887342611

El modelo tiene un rendimiento bastante pobre, solo un 56% de las predicciones son correctas. Con este resultado es incluso casi igual que lanzar una moneda al aire y determinar si el paciente muere o no, ya que las chances del lanzmaiento de la moneda son 50% (true) y 50%(false).

Se crearán nuevos modelos para comparar si alguno de ellos obtiene un mejor Score.  
***No olvidar que el modelo fue entrenado con el 100% de los datos, lo cual es un error. En futuros fix se dividirá en train y test sets.***

## 4. Decision Tree Model

### 4.1 Fit - Ajuste del modelo

In [25]:
from sksurv.tree import SurvivalTree

In [26]:
model_tree = SurvivalTree()

In [27]:
model_tree.fit(X_train, y_train)

SurvivalTree()

### 4.2 Predict - Predicciones con el modelo ajustado

In [28]:
model_tree.predict(X_test)

array([ 32.2       ,  22.        ,   0.        ,  11.        ,
         0.        ,  11.        ,   0.        ,  32.        ,
        41.75      , 102.05      ,  68.83333333,   0.        ,
         0.        ,   0.        , 102.05      ,   0.        ,
        11.        , 111.33333333,  30.        ,   0.        ,
        56.        ,  56.        ,  85.83333333,  60.33333333,
         7.33333333,   0.        , 110.5       ,  22.        ,
        55.38333333,   0.        ,  30.        ,   0.        ,
        56.        ,   7.33333333, 111.33333333,   0.        ,
       102.05      ,  30.        ,  32.        ,  33.83333333,
        55.38333333,  85.83333333,  32.2       ,  18.        ,
        30.        , 119.        ,  18.        ,  23.        ,
         0.        ,  70.5       ,   0.        ,  29.65      ,
        55.38333333,  85.83333333,  60.33333333, 102.05      ,
         0.        ,   0.        ,  23.        ,  22.        ,
       119.        ,  85.83333333,  32.        ,  22.  

### 4.3 Score - Evaluación de las predicciones

In [29]:
model_tree.score(X_test, y_test)

0.8293571901921802

## 5. Random Forest Model

### 5.1 Fit - Ajuste del modelo

In [30]:
from sksurv.ensemble import RandomSurvivalForest

In [31]:
model_rf = RandomSurvivalForest()

In [33]:
model_rf.fit(X_train, y_train)

RandomSurvivalForest()

### 5.2 Predict - Predicciones con el modelo ajustado

In [34]:
model_rf.predict(X_test)

array([47.19639557, 30.63853297, 21.16704898, 32.72531978, 17.59020789,
       18.93118519, 12.00850193, 20.14633791, 41.00247489, 85.26265873,
       44.63185945, 15.76540285, 35.08725393, 25.06022687, 82.49606268,
       20.87038232, 17.80218519, 75.12089673, 10.90974681, 11.78874076,
       42.62414166, 36.13873799, 78.71905983, 77.82080513, 14.10982247,
       29.58999143, 76.23700458, 30.65502045, 54.07858667, 23.08949143,
       33.32350645, 13.41987063, 46.32480148, 12.75808899, 69.78688171,
       11.00618015, 81.39422016,  8.9826357 , 27.26500458, 12.27046298,
       28.97302298, 71.20610127, 56.27691939, 35.37514369,  8.46096904,
       21.94112363, 23.52650154, 54.48411654, 37.70813795, 34.09139999,
       14.92729029, 23.4542238 , 23.87631385, 69.91890687, 61.53039918,
       90.35653175, 26.47866677,  5.28535089, 22.08475952,  7.90848019,
       24.11556286, 74.02898485, 15.98502045,  3.63798019, 48.9892487 ,
       61.73018436, 18.2792043 , 22.1547238 ,  9.48630237, 66.32

### 5.3 Score - Evaluación de las predicciones

In [35]:
model_rf.score(X_test, y_test)

0.87740225314778

## 6. Support Vector Machine

In [38]:
from sksurv.svm import FastSurvivalSVM

In [39]:
model_svm = FastSurvivalSVM()

model_svm.fit(X_train, y_train)
model_svm.predict(X_test)
model_svm.score(X_test, y_test)

0.7461895294897283

## 7. Resultado de los modelos

In [36]:
columnas = ["Modelo", "Score"]

In [42]:
df_resultados = pd.DataFrame({
    "Modelos": ["Cox PH", "Decision Tree", "Random Forest", "SVM"],
    "Score": [model_cox.score(X_test,y_test), model_tree.score(X_test,y_test), model_rf.score(X_test,y_test), model_svm.score(X_test,y_test)]
    }
    )
df_resultados.style.background_gradient()

,Modelos,Score
0,Cox PH,0.756130
1,Decision Tree,0.829357
2,Random Forest,0.877402
3,SVM,0.746190
